In [1]:
import pandas as pd

In [2]:
import pymysql
from sqlalchemy import create_engine

In [3]:
pymysql.install_as_MySQLdb()

## Store CSV into DataFrame

In [4]:
Food_file = "C:/Users/New User/Desktop/PREWORK_SJT/Module-14-ETL-Project/Serlos/food_production_vs_climate/worldfoodproduction.csv"

https://www.kaggle.com/dorbicycle/world-foodfeed-production

In [5]:
food_df = pd.read_csv(Food_file, encoding='latin1')
food_df.head()

,Area Abbreviation,Area Code,Area,Item Code,Item,Element Code,Element,Unit,latitude,longitude,...,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013
0,AFG,2,Afghanistan,2511,Wheat and products,5142,Food,1000 tonnes,33.94,67.71,...,3249.0,3486.0,3704.0,4164.0,4252.0,4538.0,4605.0,4711.0,4810,4895
1,AFG,2,Afghanistan,2805,Rice (Milled Equivalent),5142,Food,1000 tonnes,33.94,67.71,...,419.0,445.0,546.0,455.0,490.0,415.0,442.0,476.0,425,422
2,AFG,2,Afghanistan,2513,Barley and products,5521,Feed,1000 tonnes,33.94,67.71,...,58.0,236.0,262.0,263.0,230.0,379.0,315.0,203.0,367,360
3,AFG,2,Afghanistan,2513,Barley and products,5142,Food,1000 tonnes,33.94,67.71,...,185.0,43.0,44.0,48.0,62.0,55.0,60.0,72.0,78,89
4,AFG,2,Afghanistan,2514,Maize and products,5521,Feed,1000 tonnes,33.94,67.71,...,120.0,208.0,233.0,249.0,247.0,195.0,178.0,191.0,200,200


In [ ]:
food_df.describe()

In [ ]:
# Remove the rows with missing data
food_df = food_df.dropna(how="any")
food_df.count()

In [ ]:
food_df.columns

In [ ]:
food_df.dtypes

## Create new data set with select columns

In [6]:
food2011 = food_df[["Area Abbreviation","Area","Item Code", "Item","Element Code","latitude","longitude","Y2011","Unit"]]
food2011.head()

,Area Abbreviation,Area,Item Code,Item,Element Code,latitude,longitude,Y2011,Unit
0,AFG,Afghanistan,2511,Wheat and products,5142,33.94,67.71,4711.0,1000 tonnes
1,AFG,Afghanistan,2805,Rice (Milled Equivalent),5142,33.94,67.71,476.0,1000 tonnes
2,AFG,Afghanistan,2513,Barley and products,5521,33.94,67.71,203.0,1000 tonnes
3,AFG,Afghanistan,2513,Barley and products,5142,33.94,67.71,72.0,1000 tonnes
4,AFG,Afghanistan,2514,Maize and products,5521,33.94,67.71,191.0,1000 tonnes


In [7]:
food_2011 = food2011.loc[(food2011["Element Code"] == 5142)]
food_2011.head()

,Area Abbreviation,Area,Item Code,Item,Element Code,latitude,longitude,Y2011,Unit
0,AFG,Afghanistan,2511,Wheat and products,5142,33.94,67.71,4711.0,1000 tonnes
1,AFG,Afghanistan,2805,Rice (Milled Equivalent),5142,33.94,67.71,476.0,1000 tonnes
3,AFG,Afghanistan,2513,Barley and products,5142,33.94,67.71,72.0,1000 tonnes
5,AFG,Afghanistan,2514,Maize and products,5142,33.94,67.71,73.0,1000 tonnes
6,AFG,Afghanistan,2517,Millet and products,5142,33.94,67.71,14.0,1000 tonnes


### Connect to local database

In [8]:
connection_string = "root:toor@localhost/foodprod_db"
engine = create_engine(f'mysql://{connection_string}')

In [9]:
# Confirm tables
engine.table_names()

OperationalError: (pymysql.err.OperationalError) (2003, "Can't connect to MySQL server on 'localhost' ([WinError 10061] No connection could be made because the target machine actively refused it)") (Background on this error at: http://sqlalche.me/e/e3q8)

### Load DataFrames into database

In [ ]:
food_2011.to_sql(name='area_abb', con=engine, if_exists='append', index=True)

In [ ]:
pd.read_sql_query('select * from area_abb', con=engine).head()

In [34]:
###The precipitation data is coming from the link below: Average precipitation in depth (mm per year). 
###Average precipitation is the long-term average in depth (over space and time) of annual precipitation in the country. 
###Precipitation is defined as any kind of water that falls from clouds as a liquid or a solid.

url = 'https://www.nationmaster.com/country-info/stats/Geography/Average-precipitation-in-depth/Mm-per-year#2011'

In [41]:
tables = pd.read_html(url)
tables

[        #                                            COUNTRY   AMOUNT  DATE  \
 0       1                              Sao Tome and Principe  3200.00  2011   
 1       2                                   Papua New Guinea  3142.00  2011   
 2       3                                    Solomon Islands  3028.00  2011   
 3       4                                         Costa Rica  2926.00  2011   
 4       5                                           Malaysia  2875.00  2011   
 5       6                                             Brunei  2722.00  2011   
 6       7                                          Indonesia  2702.00  2011   
 7       8                                             Panama  2692.00  2011   
 8       9                                         Bangladesh  2666.00  2011   
 9      10                                           Colombia  2612.00  2011   
 10     11                                               Fiji  2592.00  2011   
 11     12                              

In [42]:
df = tables[0]
df.columns = ['ID','COUNTRY', 'RAIN (mm)', 'DATE', 'blank', 'blank 2']
df.head()

,ID,COUNTRY,RAIN (mm),DATE,blank,blank 2
0,1,Sao Tome and Principe,3200.0,2011,NaN,NaN
1,2,Papua New Guinea,3142.0,2011,NaN,NaN
2,3,Solomon Islands,3028.0,2011,NaN,NaN
3,4,Costa Rica,2926.0,2011,NaN,NaN
4,5,Malaysia,2875.0,2011,NaN,NaN


In [47]:
df.drop(['ID', 'blank', 'blank 2'], axis=1).head()

,COUNTRY,RAIN (mm),DATE
0,Sao Tome and Principe,3200.0,2011
1,Papua New Guinea,3142.0,2011
2,Solomon Islands,3028.0,2011
3,Costa Rica,2926.0,2011
4,Malaysia,2875.0,2011


In [48]:
Temperature_data = 'C:/Users/CAEAIZPU/OneDrive - kochind.com/Desktop/ETL Project/cckp_historical_data_0.csv'

In [49]:
Temperature_df = pd.read_csv(Temperature_data)
Temperature_df.head()

,ISO_3DIGIT,Jan_Temp,Feb_temp,Mar_temp,Apr_Temp,May_temp,Jun_Temp,July_Temp,Aug_Temp,Sept_temp,Oct_temp,Nov_Temp,Dec_temp,Annual_temp
0,AFG,0.07,2.11,7.60,13.37,18.22,23.20,25.26,23.77,19.03,12.99,7.00,2.43,12.92
1,AGO,22.58,22.68,22.78,22.35,20.74,18.37,17.95,19.90,22.19,23.18,22.79,22.61,21.51
2,ALB,2.02,3.22,6.04,9.92,14.44,17.93,20.54,20.48,17.16,12.27,7.58,3.65,11.27
3,ARE,18.43,19.43,22.61,26.58,30.62,32.46,33.80,33.55,31.74,28.34,24.06,20.28,26.83
4,ARG,20.80,19.90,17.51,14.05,10.65,7.66,7.42,9.02,11.53,14.67,17.54,19.83,14.22


In [57]:
Temperature_df.columns

Index(['ISO_3DIGIT', 'Jan_Temp', 'Feb_temp', 'Mar_temp', 'Apr_Temp',
       'May_temp', 'Jun_Temp', 'July_Temp', 'Aug_Temp', 'Sept_temp',
       'Oct_temp', 'Nov_Temp', 'Dec_temp', 'Annual_temp'],
      dtype='object')

In [58]:
Temperature_df.drop(['Jan_Temp', 'Feb_temp', 'Mar_temp', 'Apr_Temp',
       'May_temp', 'Jun_Temp', 'July_Temp', 'Aug_Temp', 'Sept_temp',
       'Oct_temp', 'Nov_Temp', 'Dec_temp'], axis=1).head()

,ISO_3DIGIT,Annual_temp
0,AFG,12.92
1,AGO,21.51
2,ALB,11.27
3,ARE,26.83
4,ARG,14.22


In [60]:
import requests as req

In [61]:
# Save config info
url = "http://api.openweathermap.org/data/2.5/weather"

params = {"appid": "fb5402f7576ca409e1bbcb49d0ba91d9",
          "units": "metric"}

# Use 'for' loop to retrieve weather info by rows in dataframe
for column, row in df.iterrows():
    
    # Reference rows for 'q' param
    params["q"] = f'{row["COUNTRY"]}'
    
    # Get and print links for each country
    print(f'Weather information for {params["q"]}')
    weather_response = req.get(url, params)
    print(weather_response.url)
    weather_response  = weather_response.json()
    
    # Get weather data and input into dataframe
    df.set_value(column, "Latitude", weather_response.get("coord", {}).get("lat"))
    df.set_value(column, "Longitude", weather_response.get("coord", {}).get("lon"))
    df.set_value(column, "Max Temperature", weather_response.get("main", {}).get("temp_max"))
    df.set_value(column, "Min Temperature", weather_response.get("main", {}).get("temp_min"))
    df.set_value(column, "Sunrise", weather_response.get("sys", {}).get("sunrise"))
    df.set_value(column, "Sunset", weather_response.get("sys", {}).get("sunset"))

Weather information for Sao Tome and Principe
http://api.openweathermap.org/data/2.5/weather?appid=fb5402f7576ca409e1bbcb49d0ba91d9&units=metric&q=Sao+Tome+and+Principe
Weather information for Papua New Guinea
http://api.openweathermap.org/data/2.5/weather?appid=fb5402f7576ca409e1bbcb49d0ba91d9&units=metric&q=Papua+New+Guinea
Weather information for Solomon Islands


C:\Users\CAEAIZPU\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:20: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
C:\Users\CAEAIZPU\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:21: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
C:\Users\CAEAIZPU\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:22: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
C:\Users\CAEAIZPU\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:23: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
C:\Users\CAEAIZPU\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:24: FutureWarning: set_v

http://api.openweathermap.org/data/2.5/weather?appid=fb5402f7576ca409e1bbcb49d0ba91d9&units=metric&q=Solomon+Islands
Weather information for Costa Rica
http://api.openweathermap.org/data/2.5/weather?appid=fb5402f7576ca409e1bbcb49d0ba91d9&units=metric&q=Costa+Rica
Weather information for Malaysia
http://api.openweathermap.org/data/2.5/weather?appid=fb5402f7576ca409e1bbcb49d0ba91d9&units=metric&q=Malaysia
Weather information for Brunei
http://api.openweathermap.org/data/2.5/weather?appid=fb5402f7576ca409e1bbcb49d0ba91d9&units=metric&q=Brunei
Weather information for Indonesia
http://api.openweathermap.org/data/2.5/weather?appid=fb5402f7576ca409e1bbcb49d0ba91d9&units=metric&q=Indonesia
Weather information for Panama
http://api.openweathermap.org/data/2.5/weather?appid=fb5402f7576ca409e1bbcb49d0ba91d9&units=metric&q=Panama
Weather information for Bangladesh
http://api.openweathermap.org/data/2.5/weather?appid=fb5402f7576ca409e1bbcb49d0ba91d9&units=metric&q=Bangladesh
Weather information for

http://api.openweathermap.org/data/2.5/weather?appid=fb5402f7576ca409e1bbcb49d0ba91d9&units=metric&q=Madagascar
Weather information for East Timor
http://api.openweathermap.org/data/2.5/weather?appid=fb5402f7576ca409e1bbcb49d0ba91d9&units=metric&q=East+Timor
Weather information for Nepal
http://api.openweathermap.org/data/2.5/weather?appid=fb5402f7576ca409e1bbcb49d0ba91d9&units=metric&q=Nepal
Weather information for Albania
http://api.openweathermap.org/data/2.5/weather?appid=fb5402f7576ca409e1bbcb49d0ba91d9&units=metric&q=Albania
Weather information for Haiti
http://api.openweathermap.org/data/2.5/weather?appid=fb5402f7576ca409e1bbcb49d0ba91d9&units=metric&q=Haiti
Weather information for Saint Kitts and Nevis
http://api.openweathermap.org/data/2.5/weather?appid=fb5402f7576ca409e1bbcb49d0ba91d9&units=metric&q=Saint+Kitts+and+Nevis
Weather information for Barbados
http://api.openweathermap.org/data/2.5/weather?appid=fb5402f7576ca409e1bbcb49d0ba91d9&units=metric&q=Barbados
Weather inform

http://api.openweathermap.org/data/2.5/weather?appid=fb5402f7576ca409e1bbcb49d0ba91d9&units=metric&q=Group+of+7+countries+%28G7%29+average++%28profile%29
Weather information for Luxembourg
http://api.openweathermap.org/data/2.5/weather?appid=fb5402f7576ca409e1bbcb49d0ba91d9&units=metric&q=Luxembourg
Weather information for Cold countries average  (profile)
http://api.openweathermap.org/data/2.5/weather?appid=fb5402f7576ca409e1bbcb49d0ba91d9&units=metric&q=Cold+countries+average++%28profile%29
Weather information for Comoros
http://api.openweathermap.org/data/2.5/weather?appid=fb5402f7576ca409e1bbcb49d0ba91d9&units=metric&q=Comoros
Weather information for France
http://api.openweathermap.org/data/2.5/weather?appid=fb5402f7576ca409e1bbcb49d0ba91d9&units=metric&q=France
Weather information for failed states average  (profile)
http://api.openweathermap.org/data/2.5/weather?appid=fb5402f7576ca409e1bbcb49d0ba91d9&units=metric&q=failed+states+average++%28profile%29
Weather information for Non

http://api.openweathermap.org/data/2.5/weather?appid=fb5402f7576ca409e1bbcb49d0ba91d9&units=metric&q=South+Africa
Weather information for Pakistan
http://api.openweathermap.org/data/2.5/weather?appid=fb5402f7576ca409e1bbcb49d0ba91d9&units=metric&q=Pakistan
Weather information for Russia
http://api.openweathermap.org/data/2.5/weather?appid=fb5402f7576ca409e1bbcb49d0ba91d9&units=metric&q=Russia
Weather information for OPEC countries average  (profile)
http://api.openweathermap.org/data/2.5/weather?appid=fb5402f7576ca409e1bbcb49d0ba91d9&units=metric&q=OPEC+countries+average++%28profile%29
Weather information for Moldova
http://api.openweathermap.org/data/2.5/weather?appid=fb5402f7576ca409e1bbcb49d0ba91d9&units=metric&q=Moldova
Weather information for Azerbaijan
http://api.openweathermap.org/data/2.5/weather?appid=fb5402f7576ca409e1bbcb49d0ba91d9&units=metric&q=Azerbaijan
Weather information for Israel
http://api.openweathermap.org/data/2.5/weather?appid=fb5402f7576ca409e1bbcb49d0ba91d9&un

In [62]:
df.head()

,ID,COUNTRY,RAIN (mm),DATE,blank,blank 2,Latitude,Longitude,Max Temperature,Min Temperature,Sunrise,Sunset
0,1,Sao Tome and Principe,3200.0,2011,NaN,NaN,1.00,7.00,27.21,27.21,1.542345e+09,1.542389e+09
1,2,Papua New Guinea,3142.0,2011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Solomon Islands,3028.0,2011,NaN,NaN,-8.00,159.00,28.81,28.81,1.542308e+09,1.542353e+09
3,4,Costa Rica,2926.0,2011,NaN,NaN,24.59,-107.39,30.00,30.00,1.542375e+09,1.542414e+09
4,5,Malaysia,2875.0,2011,NaN,NaN,2.50,112.50,25.00,25.00,1.542320e+09,1.542363e+09


In [63]:
new_df = df.drop(['ID', 'DATE', 'blank', 'blank 2'], axis=1).head()

In [64]:
new_df.head()

,COUNTRY,RAIN (mm),Latitude,Longitude,Max Temperature,Min Temperature,Sunrise,Sunset
0,Sao Tome and Principe,3200.0,1.00,7.00,27.21,27.21,1.542345e+09,1.542389e+09
1,Papua New Guinea,3142.0,NaN,NaN,NaN,NaN,NaN,NaN
2,Solomon Islands,3028.0,-8.00,159.00,28.81,28.81,1.542308e+09,1.542353e+09
3,Costa Rica,2926.0,24.59,-107.39,30.00,30.00,1.542375e+09,1.542414e+09
4,Malaysia,2875.0,2.50,112.50,25.00,25.00,1.542320e+09,1.542363e+09


In [102]:
import time


In [110]:
new_df.dropna()

,COUNTRY,RAIN (mm),Latitude,Longitude,Max Temperature,Min Temperature,Sunrise,Sunset
0,Sao Tome and Principe,3200.0,1.00,7.00,27.21,27.21,1.542345e+09,1.542389e+09
2,Solomon Islands,3028.0,-8.00,159.00,28.81,28.81,1.542308e+09,1.542353e+09
3,Costa Rica,2926.0,24.59,-107.39,30.00,30.00,1.542375e+09,1.542414e+09
4,Malaysia,2875.0,2.50,112.50,25.00,25.00,1.542320e+09,1.542363e+09


In [111]:
sunrise = new_df['Sunrise'].dropna().apply(time.gmtime)
sunrise

0     (2018, 11, 16, 5, 14, 31, 4, 320, 0)
2    (2018, 11, 15, 18, 54, 12, 3, 319, 0)
3    (2018, 11, 16, 13, 26, 16, 4, 320, 0)
4    (2018, 11, 15, 22, 14, 29, 3, 319, 0)
Name: Sunrise, dtype: object

In [112]:
sunset = new_df['Sunset'].dropna().apply(time.gmtime)
sunset

0    (2018, 11, 16, 17, 18, 56, 4, 320, 0)
2     (2018, 11, 16, 7, 23, 12, 4, 320, 0)
3     (2018, 11, 17, 0, 22, 12, 5, 321, 0)
4    (2018, 11, 16, 10, 14, 50, 4, 320, 0)
Name: Sunset, dtype: object

In [ ]:
Weather_df = Weather_df[["COUNTRY","RAIN (mm)","Latitude","Longitud",
                                "Max Temperature","Min Temperature","% Passing Math","% Passing Reading",
                                 "Overall Passing Rate"]]